In [1]:
# Dependencies
import numpy as np
from numpy import genfromtxt
from time import time
from datetime import datetime
from sqlalchemy import Column, Integer, Float, Date, String, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.orm import Session
from sqlalchemy import Index
from sqlalchemy import MetaData
from sqlalchemy import Table
import csv
import pandas as pd
from sqlalchemy.ext.automap import automap_base
from sqlalchemy import create_engine, inspect
from sqlalchemy import func

In [2]:
measurement = "./Resources/hawaii_measurements.csv"
measurement_df = pd.read_csv(measurement)
measurement_df.head()

,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0.00,63
2,USC00519397,2010-01-03,0.00,74
3,USC00519397,2010-01-04,0.00,76
4,USC00519397,2010-01-06,NaN,73


In [3]:
station = "./Resources/hawaii_stations.csv"
station_df = pd.read_csv(station)
station_df.head()

,station,name,latitude,longitude,elevation
0,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0
1,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.5213,-157.8374,7.0
3,USC00517948,"PEARL CITY, HI US",21.3934,-157.9751,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.4992,-158.0111,306.6


In [4]:
clean_measurement_df = measurement_df.dropna(how='any')
clean_measurement_df.head()

,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0.00,63
2,USC00519397,2010-01-03,0.00,74
3,USC00519397,2010-01-04,0.00,76
5,USC00519397,2010-01-07,0.06,70


In [5]:
#clean_measurement_df.drop(clean_measurement_df.columns[0], axis=1, inplace=True)
#clean_measurement_df.head()

In [6]:
clean_measurement_df.to_csv("./Resources/withoutid_clean_measurement_df.csv", encoding="utf-8", index=False)

In [7]:
clean_station_df = station_df.dropna(how='any')
clean_station_df.head()

,station,name,latitude,longitude,elevation
0,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0
1,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.5213,-157.8374,7.0
3,USC00517948,"PEARL CITY, HI US",21.3934,-157.9751,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.4992,-158.0111,306.6


In [8]:
#clean_station_df.drop(clean_station_df.columns[0], axis=1, inplace=True)

#clean_station_df.head()

In [9]:
clean_station_df = clean_station_df.rename(columns={"name":"station_name"})
clean_station_df

,station,station_name,latitude,longitude,elevation
0,USC00519397,"WAIKIKI 717.2, HI US",21.27160,-157.81680,3.0
1,USC00513117,"KANEOHE 838.1, HI US",21.42340,-157.80150,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.52130,-157.83740,7.0
3,USC00517948,"PEARL CITY, HI US",21.39340,-157.97510,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.49920,-158.01110,306.6
5,USC00519523,"WAIMANALO EXPERIMENTAL FARM, HI US",21.33556,-157.71139,19.5
6,USC00519281,"WAIHEE 837.5, HI US",21.45167,-157.84889,32.9
7,USC00511918,"HONOLULU OBSERVATORY 702.2, HI US",21.31520,-157.99920,0.9
8,USC00516128,"MANOA LYON ARBO 785.2, HI US",21.33310,-157.80250,152.4


In [10]:
clean_station_df.to_csv("./Resources/withoutid_clean_station_df.csv", encoding="utf-8", index=False)

In [11]:
clean_measurement = "resources/withoutid_clean_measurement_df.csv"
clean_measurement_df = pd.read_csv(clean_measurement )
clean_measurement_df.head()

,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0.00,63
2,USC00519397,2010-01-03,0.00,74
3,USC00519397,2010-01-04,0.00,76
4,USC00519397,2010-01-07,0.06,70


In [12]:
clean_station = "resources/withoutid_clean_station_df.csv"
clean_station_df = pd.read_csv(clean_station)

clean_station_df.head()

,station,station_name,latitude,longitude,elevation
0,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0
1,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.5213,-157.8374,7.0
3,USC00517948,"PEARL CITY, HI US",21.3934,-157.9751,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.4992,-158.0111,306.6


In [13]:
clean_station_df.dtypes

station          object
station_name     object
latitude        float64
longitude       float64
elevation       float64
dtype: object

In [14]:
Base = declarative_base()
class Measurement(Base):
    __tablename__ = 'measurement_table'
    __table_args__ = {'sqlite_autoincrement': True}
    __table_args__ = {'extend_existing': True}
    id = Column(Integer, primary_key=True, nullable=False)
    station = Column(String)
    date = Column(Date)
    prcp = Column(Float)
    tobs = Column(Float)


engine = create_engine("sqlite:///hawaii.sqlite")
Base.metadata.create_all(engine)

#Create the session
session = Session(bind=engine)

In [15]:
for i, row in clean_measurement_df.iterrows():
    
    record = Measurement(**{
        'station' : row.station,
        'date' : datetime.strptime(row.date, '%Y-%m-%d').date(),
        'prcp' : row.prcp,
        'tobs' : row.tobs
})

    session.add(record)
session.commit()

In [16]:
row

station    USC00516128
date        2017-08-23
prcp              0.45
tobs                76
Name: 18102, dtype: object

In [17]:
Base = declarative_base()
class Station (Base):
    __tablename__ = 'station_table'
    __table_args__ = {'sqlite_autoincrement': True}

    id = Column(Integer, primary_key=True, nullable=False)
    station = Column(String(225))
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)


engine = create_engine("sqlite:///hawaii.sqlite")
Base.metadata.create_all(engine)

#Create the session
session = Session(bind=engine)

In [18]:
for i, row in clean_station_df.iterrows():
    record = Station(**{
         'station': row.station_name,
         'latitude': row.latitude,
         'longitude' : row.longitude,
         'elevation': row.elevation,
})
    print (row.station_name)
    session.add(record)
session.commit()

WAIKIKI 717.2, HI US
KANEOHE 838.1, HI US
KUALOA RANCH HEADQUARTERS 886.9, HI US
PEARL CITY, HI US
UPPER WAHIAWA 874.3, HI US
WAIMANALO EXPERIMENTAL FARM, HI US
WAIHEE 837.5, HI US
HONOLULU OBSERVATORY 702.2, HI US
MANOA LYON ARBO 785.2, HI US


In [19]:
row

station                          USC00516128
station_name    MANOA LYON ARBO 785.2, HI US
latitude                             21.3331
longitude                           -157.803
elevation                              152.4
Name: 8, dtype: object

In [20]:
Base = declarative_base()

In [21]:
engine = create_engine("sqlite:///hawaii.sqlite")
Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()

['measurement', 'measurement_table', 'station', 'station_table']

In [22]:
Measurement=Base.classes.measurement_table
Measurement

sqlalchemy.ext.automap.measurement_table

In [23]:
Station=Base.classes.station_table
Station

sqlalchemy.ext.automap.station_table

In [24]:
inspector = inspect(engine)
inspector.get_table_names()

['measurement',
 'measurement_table',
 'sqlite_sequence',
 'station',
 'station_table']

In [25]:
columns = inspector.get_columns('station_table')
for c in columns:
    print(c['name'], c["type"])

id INTEGER
station VARCHAR(225)
latitude FLOAT
longitude FLOAT
elevation FLOAT


In [26]:
from sqlalchemy import inspect

mapper = inspect(Measurement)
print(mapper)

Mapper|measurement_table|measurement_table


In [27]:
from sqlalchemy import inspect

mapper = inspect(Station)
print(mapper)

Mapper|station_table|station_table


In [28]:
Session = sessionmaker(bind=engine)
# Session is a class
session = Session()
# now session is a instance of the class Session

In [29]:
engine.execute('SELECT * FROM station_table LIMIT 5').fetchall()

[(1, 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (2, 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 (3, 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 (4, 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 (5, 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6)]

In [30]:
engine.execute('SELECT * FROM measurement_table LIMIT 5').fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65.0),
 (2, 'USC00519397', '2010-01-02', 0.0, 63.0),
 (3, 'USC00519397', '2010-01-03', 0.0, 74.0),
 (4, 'USC00519397', '2010-01-04', 0.0, 76.0),
 (5, 'USC00519397', '2010-01-07', 0.06, 70.0)]